# Download

In [1]:
import os
import base64
import requests
import json
import base64 
import tempfile
from urllib.parse import urlparse
from urllib.parse import unquote

import pandas as pd
import numpy as np
from getpass import getpass
from glob import glob
from tqdm import tqdm
pd.set_option('display.max_columns', None)

access_token = 'ALtAyMZA.Y1sITQvTj6OXSr6XUz31P2ov3XvsdkhpbijDutzY0glCVNohhzM4okEe'

In [8]:
fname = r'D:\OneDrive_Emory\OneDrive - Emory\Research_doc\hourly_population\github\split_population\2022\07\12086.csv.gz'

df = pd.read_csv(fname)
df['CBG'] = df['CBG'].astype(str).str.zfill(12)
CBG_list = ['120860099031', '120860099042'  ]

CBG_list

df_sub = df[df['CBG'].isin(CBG_list)]
df_sub

,CBG,2022-07-01 00:00:00,2022-07-01 01:00:00,2022-07-01 02:00:00,2022-07-01 03:00:00,2022-07-01 04:00:00,2022-07-01 05:00:00,2022-07-01 06:00:00,2022-07-01 07:00:00,2022-07-01 08:00:00,2022-07-01 09:00:00,2022-07-01 10:00:00,2022-07-01 11:00:00,2022-07-01 12:00:00,2022-07-01 13:00:00,2022-07-01 14:00:00,2022-07-01 15:00:00,2022-07-01 16:00:00,2022-07-01 17:00:00,2022-07-01 18:00:00,2022-07-01 19:00:00,2022-07-01 20:00:00,2022-07-01 21:00:00,2022-07-01 22:00:00,2022-07-01 23:00:00,2022-07-02 00:00:00,2022-07-02 01:00:00,2022-07-02 02:00:00,2022-07-02 03:00:00,2022-07-02 04:00:00,2022-07-02 05:00:00,2022-07-02 06:00:00,2022-07-02 07:00:00,2022-07-02 08:00:00,2022-07-02 09:00:00,2022-07-02 10:00:00,2022-07-02 11:00:00,2022-07-02 12:00:00,2022-07-02 13:00:00,2022-07-02 14:00:00,2022-07-02 15:00:00,2022-07-02 16:00:00,2022-07-02 17:00:00,2022-07-02 18:00:00,2022-07-02 19:00:00,2022-07-02 20:00:00,2022-07-02 21:00:00,2022-07-02 22:00:00,2022-07-02 23:00:00,2022-07-03 00:00:00,2022-07-03 01:00:00,2022-07-03 02:00:00,2022-07-03 03:00:00,2022-07-03 04:00:00,2022-07-03 05:00:00,2022-07-03 06:00:00,2022-07-03 07:00:00,2022-07-03 08:00:00,2022-07-03 09:00:00,2022-07-03 10:00:00,2022-07-03 11:00:00,2022-07-03 12:00:00,2022-07-03 13:00:00,2022-07-03 14:00:00,2022-07-03 15:00:00,2022-07-03 16:00:00,2022-07-03 17:00:00,2022-07-03 18:00:00,2022-07-03 19:00:00,2022-07-03 20:00:00,2022-07-03 21:00:00,2022-07-03 22:00:00,2022-07-03 23:00:00,2022-07-04 00:00:00,2022-07-04 01:00:00,2022-07-04 02:00:00,2022-07-04 03:00:00,2022-07-04 04:00:00,2022-07-04 05:00:00,2022-07-04 06:00:00,2022-07-04 07:00:00,2022-07-04 08:00:00,2022-07-04 09:00:00,2022-07-04 10:00:00,2022-07-04 11:00:00,2022-07-04 12:00:00,2022-07-04 13:00:00,2022-07-04 14:00:00,2022-07-04 15:00:00,2022-07-04 16:00:00,2022-07-04 17:00:00,2022-07-04 18:00:00,2022-07-04 19:00:00,2022-07-04 20:00:00,2022-07-04 21:00:00,2022-07-04 22:00:00,2022-07-04 23:00:00,2022-07-05 00:00:00,2022-07-05 01:00:00,2022-07-05 02:00:00,2022-07-05 03:00:00,2022-07-05 04:00:00,2022-07-05 05:00:00,2022-07-05 06:00:00,2022-07-05 07:00:00,2022-07-05 08:00:00,2022-07-05 09:00:00,2022-07-05 10:00:00,2022-07-05 11:00:00,2022-07-05 12:00:00,2022-07-05 13:00:00,2022-07-05 14:00:00,2022-07-05 15:00:00,2022-07-05 16:00:00,2022-07-05 17:00:00,2022-07-05 18:00:00,2022-07-05 19:00:00,2022-07-05 20:00:00,2022-07-05 21:00:00,2022-07-05 22:00:00,2022-07-05 23:00:00,2022-07-06 00:00:00,2022-07-06 01:00:00,2022-07-06 02:00:00,2022-07-06 03:00:00,2022-07-06 04:00:00,2022-07-06 05:00:00,2022-07-06 06:00:00,2022-07-06 07:00:00,2022-07-06 08:00:00,2022-07-06 09:00:00,2022-07-06 10:00:00,2022-07-06 11:00:00,2022-07-06 12:00:00,2022-07-06 13:00:00,2022-07-06 14:00:00,2022-07-06 15:00:00,2022-07-06 16:00:00,2022-07-06 17:00:00,2022-07-06 18:00:00,2022-07-06 19:00:00,2022-07-06 20:00:00,2022-07-06 21:00:00,2022-07-06 22:00:00,2022-07-06 23:00:00,2022-07-07 00:00:00,2022-07-07 01:00:00,2022-07-07 02:00:00,2022-07-07 03:00:00,2022-07-07 04:00:00,2022-07-07 05:00:00,2022-07-07 06:00:00,2022-07-07 07:00:00,2022-07-07 08:00:00,2022-07-07 09:00:00,2022-07-07 10:00:00,2022-07-07 11:00:00,2022-07-07 12:00:00,2022-07-07 13:00:00,2022-07-07 14:00:00,2022-07-07 15:00:00,2022-07-07 16:00:00,2022-07-07 17:00:00,2022-07-07 18:00:00,2022-07-07 19:00:00,2022-07-07 20:00:00,2022-07-07 21:00:00,2022-07-07 22:00:00,2022-07-07 23:00:00,2022-07-08 00:00:00,2022-07-08 01:00:00,2022-07-08 02:00:00,2022-07-08 03:00:00,2022-07-08 04:00:00,2022-07-08 05:00:00,2022-07-08 06:00:00,2022-07-08 07:00:00,2022-07-08 08:00:00,2022-07-08 09:00:00,2022-07-08 10:00:00,2022-07-08 11:00:00,2022-07-08 12:00:00,2022-07-08 13:00:00,2022-07-08 14:00:00,2022-07-08 15:00:00,2022-07-08 16:00:00,2022-07-08 17:00:00,2022-07-08 18:00:00,2022-07-08 19:00:00,2022-07-08 20:00:00,2022-07-08 21:00:00,2022-07-08 22:00:00,2022-07-08 23:00:00,2022-07-09 00:00:00,2022-07-09 01:00:00,2022-07-09 02:00:00,2022-07-09 03:00:00,2022-07-09 04:00:00,2022-07-09 05:00:00,2022-07-09 06:00:00,2022-07-09 07:0

# Extract Neighborhood patterns

In [33]:
data_dir = r"K:\SafeGraph\Advan_2023_API\Monthly_Patterns"
save_dir = r'K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis'

os.makedirs(save_dir, exist_ok=True)

In [34]:
all_files = glob(os.path.join(data_dir, '*', '*', '*', "*.gz"))
print("File count:", len(all_files))
print(all_files[0])
print(all_files[-1])

File count: 2859
K:\SafeGraph\Advan_2023_API\Monthly_Patterns\2019\01\01\Monthly_Patterns_Foot_Traffic-0-DATE_RANGE_START-2019-01-01.csv.gz
K:\SafeGraph\Advan_2023_API\Monthly_Patterns\2023\09\01\Monthly_Patterns_Foot_Traffic-858-DATE_RANGE_START-2023-09-01.csv.gz


In [35]:
# Sicheng Wang
# save_dir = r'K:\SafeGraph\Advan_2023_API\Monthly_Patterns_South_Carolina'

# countries = ['US']
# target_region = ['SC']
# mask = np.logical_and(t_df['NAICS_CODE'].astype(str) < "623", t_df['NAICS_CODE'].astype(str) > "619" ) 

# Xiao Huang
countries = ['US']
target_region = ['MN']

In [36]:
df_list = []
# use_cols = ['PLACEKEY', 'TOP_CATEGORY', 'SUB_CATEGORY', 'NAICS_CODE', 'CITY', 'REGION', 'POSTAL_CODE', 'ISO_COUNTRY_CODE']

for idx, f in enumerate(all_files):

    try:
        print(f"Processing: {idx + 1} / {len(all_files)}, ", os.path.basename(f))
        # df = pd.read_csv(f, usecols=use_cols)
        year = f.split(os.sep)[-4]
        month = f.split(os.sep)[-3]
    
        # print(year, month)
        if year < "2023":
            continue
        # break
    
        df = pd.read_csv(f)
        t_df = df[df['ISO_COUNTRY_CODE'].isin(countries)] 
        ## t_df = df[df['NAICS_CODE'].isin(target_NAICS)]
        # # t_df = t_df[t_df['CITY'].isin(target_city)]
        t_df = t_df[t_df['REGION'].isin(target_region)]
        # mask = np.logical_and(t_df['NAICS_CODE'].astype(str) < "623", t_df['NAICS_CODE'].astype(str) > "619" )  # Sicheng Wang
        mask =  t_df['NAICS_CODE'] == 712190    # Huang, Xiao
        t_df = t_df[mask]
        print(f"      Found row count: {len(t_df)}")
    
        basename = os.path.basename(f)
    
        f_dir = os.path.join(save_dir, year, month)
        os.makedirs(f_dir, exist_ok=True)
    
        fname = os.path.join(f_dir, basename)
    
        t_df.to_csv(fname, index=False)
    
        print('Saved at:', fname)

    except Exception as e:
        print("Error:", f, e)
        continue

    
    # break

# df_all = pd.concat(df_list)
# fname = os.path.join(save_dir, 'South_Carolina_POI_20240107.csv')
# df_all.to_csv(fname, index=False)
# # print("Saved at:", fname)
# df_all
df

Processing: 1 / 2859,  Monthly_Patterns_Foot_Traffic-0-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 2 / 2859,  Monthly_Patterns_Foot_Traffic-1-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 3 / 2859,  Monthly_Patterns_Foot_Traffic-10-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 4 / 2859,  Monthly_Patterns_Foot_Traffic-11-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 5 / 2859,  Monthly_Patterns_Foot_Traffic-12-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 6 / 2859,  Monthly_Patterns_Foot_Traffic-13-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 7 / 2859,  Monthly_Patterns_Foot_Traffic-14-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 8 / 2859,  Monthly_Patterns_Foot_Traffic-15-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 9 / 2859,  Monthly_Patterns_Foot_Traffic-16-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 10 / 2859,  Monthly_Patterns_Foot_Traffic-17-DATE_RANGE_START-2019-01-01.csv.gz
Processing: 11 / 2859,  Monthly_Patterns_Foot_Traffic-18-DATE_RANGE_START-2019-01-01.csv.gz

C:\Users\A\AppData\Local\Temp\ipykernel_20140\3719000839.py:17: DtypeWarning: Columns (34,36,37,38,39,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


      Found row count: 106
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\01\Monthly_Patterns_Foot_Traffic-430-DATE_RANGE_START-2023-01-01.csv.gz
Processing: 2432 / 2859,  Monthly_Patterns_Foot_Traffic-431-DATE_RANGE_START-2023-01-01.csv.gz
      Found row count: 104
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\01\Monthly_Patterns_Foot_Traffic-431-DATE_RANGE_START-2023-01-01.csv.gz
Processing: 2433 / 2859,  Monthly_Patterns_Foot_Traffic-432-DATE_RANGE_START-2023-02-01.csv.gz
      Found row count: 82
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-432-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2434 / 2859,  Monthly_Patterns_Foot_Traffic-433-DATE_RANGE_START-2023-02-01.csv.gz
      Found row count: 113
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-433-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2435 / 2859,  Monthl

C:\Users\A\AppData\Local\Temp\ipykernel_20140\3719000839.py:17: DtypeWarning: Columns (34,36,37,38,39,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


      Found row count: 95
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-449-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2451 / 2859,  Monthly_Patterns_Foot_Traffic-450-DATE_RANGE_START-2023-02-01.csv.gz
      Found row count: 102
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-450-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2452 / 2859,  Monthly_Patterns_Foot_Traffic-451-DATE_RANGE_START-2023-02-01.csv.gz
      Found row count: 122
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-451-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2453 / 2859,  Monthly_Patterns_Foot_Traffic-452-DATE_RANGE_START-2023-02-01.csv.gz
      Found row count: 102
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\02\Monthly_Patterns_Foot_Traffic-452-DATE_RANGE_START-2023-02-01.csv.gz
Processing: 2454 / 2859,  Monthl

...

C:\Users\A\AppData\Local\Temp\ipykernel_20140\3719000839.py:17: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


      Found row count: 49
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-776-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2778 / 2859,  Monthly_Patterns_Foot_Traffic-777-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 55
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-777-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2779 / 2859,  Monthly_Patterns_Foot_Traffic-778-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 110
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-778-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2780 / 2859,  Monthly_Patterns_Foot_Traffic-779-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 89
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-779-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2781 / 2859,  Monthly_

C:\Users\A\AppData\Local\Temp\ipykernel_20140\3719000839.py:17: DtypeWarning: Columns (2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


      Found row count: 93
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-792-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2794 / 2859,  Monthly_Patterns_Foot_Traffic-793-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 83
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-793-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2795 / 2859,  Monthly_Patterns_Foot_Traffic-794-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 57
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-794-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2796 / 2859,  Monthly_Patterns_Foot_Traffic-795-DATE_RANGE_START-2023-08-01.csv.gz
      Found row count: 95
Saved at: K:\SafeGraph\Advan_2023_API\Monthly_Patterns_Minneapolis\2023\08\Monthly_Patterns_Foot_Traffic-795-DATE_RANGE_START-2023-08-01.csv.gz
Processing: 2797 / 2859,  Monthly_P

,PLACEKEY,PARENT_PLACEKEY,SAFEGRAPH_BRAND_IDS,LOCATION_NAME,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,REGION,POSTAL_CODE,OPEN_HOURS,CATEGORY_TAGS,OPENED_ON,CLOSED_ON,TRACKING_CLOSED_SINCE,WEBSITES,GEOMETRY_TYPE,POLYGON_WKT,POLYGON_CLASS,ENCLOSED,PHONE_NUMBER,IS_SYNTHETIC,INCLUDES_PARKING_LOT,ISO_COUNTRY_CODE,WKT_AREA_SQ_METERS,DATE_RANGE_START,DATE_RANGE_END,RAW_VISIT_COUNTS,RAW_VISITOR_COUNTS,VISITS_BY_DAY,POI_CBG,VISITOR_HOME_CBGS,VISITOR_HOME_AGGREGATION,VISITOR_DAYTIME_CBGS,VISITOR_COUNTRY_OF_ORIGIN,DISTANCE_FROM_HOME,MEDIAN_DWELL,BUCKETED_DWELL_TIMES,RELATED_SAME_DAY_BRAND,RELATED_SAME_MONTH_BRAND,POPULARITY_BY_HOUR,POPULARITY_BY_DAY,DEVICE_TYPE,NORMALIZED_VISITS_BY_STATE_SCALING,NORMALIZED_VISITS_BY_REGION_NAICS_VISITS,NORMALIZED_VISITS_BY_REGION_NAICS_VISITORS,NORMALIZED_VISITS_BY_TOTAL_VISITS,NORMALIZED_VISITS_BY_TOTAL_VISITORS
0,zzw-222@5xc-kh3-bff,NaN,NaN,Btl,NaN,NaN,"Accounting, Tax Preparation, Bookkeeping, and ...",NaN,5412,45.493213,-122.784401,10445 SW Canyon Rd Ste 270,Beaverton,OR,97005.0,"{ ""Mon"": [[""9:30"", ""17:00""]], ""Tue"": [[""9:30"",...","Bookkeepers,Payroll",NaN,NaN,NaN,btlcompany.com,POLYGON,POLYGON ((-122.78421194699996 45.4932733930000...,SHARED_POLYGON,False,1.971345e+10,False,False,US,990.0,2023-09-01 00:00:00.000,2023-10-01 00:00:00.000,148.0,128.0,"[5,5,4,5,5,6,6,6,4,5,6,6,5,5,5,4,5,4,5,5,4,5,5...",4.106703e+11,"{""410510032002"":34,""410670302004"":17,""41067031...","{""41051003200"":34,""41067030102"":17,""4106703180...","{""410510032002"":17,""410510106003"":17,""41067031...","{""US"":102}",7335.0,49.0,"{ ""<5"":124, ""5-20"":6, ""21-60"":6, ""61-240"":10, ...","{""Honda"":23,""The Barbers"":23,""Wingstop"":23,""Ch...","{""Chevron"":90,""Fred Meyer"":80,""Chase"":77,""Fred...","[0,0,0,0,0,0,4,0,0,17,52,23,16,16,17,9,33,7,3,...","{""Monday"":20,""Tuesday"":20,""Wednesday"":21,""Thur...","{""android"":81,""ios"":4}",1793.0,0.016479,0.040054,0.000103,0.001422
1,22f-222@8ts-zhz-y5f,NaN,NaN,Abp Accounting & Tax Service,NaN,NaN,"Accounting, Tax Preparation, Bookkeeping, and ...",NaN,5412,33.378848,-111.905932,1834 E Baseline Rd Ste 104,Tempe,AZ,85283.0,NaN,Accountants,NaN,NaN,NaN,NaN,POLYGON,POLYGON ((-111.90590027599995 33.3789012660000...,SHARED_POLYGON,False,1.480838e+10,False,False,US,809.0,2023-09-01 00:00:00.000,2023-10-01 00:00:00.000,152.0,152.0,"[5,5,4,5,6,5,6,6,6,5,6,6,5,5,5,5,5,5,5,5,5,5,5...",4.013319e+10,"{""040133194022"":31,""180890122001"":16,""04021000...","{""04013319402"":31,""04013320001"":16,""0402100020...","{""040133194022"":31,""040133196003"":16,""04013319...","{""US"":122}",4621.0,5.0,"{ ""<5"":141, ""5-20"":6, ""21-60"":3, ""61-240"":0, ""...","{""CVS Pharmacy"":51,""Target"":51,""Fry's Food & D...","{""CVS Pharmacy"":110,""QuikTrip"":110,""Walmart"":1...","[0,0,0,0,4,0,0,34,21,10,24,10,27,30,10,36,21,1...","{""Monday"":21,""Tuesday"":22,""Wednesday"":20,""Thur...","{""android"":97,""ios"":24}",1839.0,0.007552,0.013226,0.000046,0.000511
2,2b2-223@62k-rds-tgk,NaN,NaN,Bollus Lynch,NaN,NaN,"Accounting, Tax Preparation, Bookkeeping, and ...",NaN,5412,42.263333,-71.789946,89 Shrewsbury St,Worcester,MA,1604.0,NaN,Accountants,NaN,NaN,NaN,bolluslynch.com,POLYGON,POLYGON ((-71.78997975323249 42.26319487816585...,OWNED_POLYGON,False,1.508756e+10,False,False,US,307.0,2023-09-01 00:00:00.000,2023-10-01 00:00:00.000,154.0,146.0,"[5,5,5,6,6,6,6,4,6,6,6,6,6,6,6,6,3,0,6,4,6,6,6...",2.502773e+11,"{""250277310023"":23,""250092059003"":23,""25009260...","{""25027731002"":46,""23011012000"":23,""2502773120...","{""250277305002"":23,""250277365005"":23,""25025071...","{""US"":114}",18687.0,22.0,"{ ""<5"":137, ""5-20"":0, ""21-60"":12, ""61-240"":4, ...","{""Berkshire Bank"":16,""Citizens Bank"":16,""Mobil...","{""Dunkin'"":108,""Mobil"":95,""CVS Pharmacy"":87,""C...","[0,0,0,0,0,0,0,3,10,6,9,7,21,25,12,53,38,11,7,...","{""Monday"":12,""Tuesday"":24,""Wednesday"":22,""Thur...","{""android"":62,""ios"":54}",1860.0,0.005419,0.014210,0.000040

In [37]:
# t_df

In [28]:
df.dtypes

PLACEKEY                                       object
PARENT_PLACEKEY                                object
SAFEGRAPH_BRAND_IDS                            object
LOCATION_NAME                                  object
BRANDS                                         object
STORE_ID                                      float64
TOP_CATEGORY                                   object
SUB_CATEGORY                                   object
NAICS_CODE                                      int64
LATITUDE                                      float64
LONGITUDE                                     float64
STREET_ADDRESS                                 object
CITY                                           object
REGION                                         object
POSTAL_CODE                                   float64
OPEN_HOURS                                     object
CATEGORY_TAGS                                  object
OPENED_ON                                      object
CLOSED_ON                   

In [23]:
STOP

NameError: name 'STOP' is not defined

## helper function

In [17]:
def safe_file_write(file_path, content):
    # Create a temporary file in the same directory as the target file
    dir_name = os.path.dirname(file_path)
    with tempfile.NamedTemporaryFile(mode='wb', dir=dir_name, delete=False) as temp_file:
        try:
            # Write the content to the temporary file
            temp_file.write(content)
            temp_file.flush()
            os.fsync(temp_file.fileno())
            temp_file.close()

            # Rename the temporary file to the target file (atomic operation)
            os.replace(temp_file.name, file_path)
        except Exception as e:
            # Handle the exception (e.g., log it, raise it, or silently ignore it)
            print(f"An error occurred while writing the file: {e}")
            os.unlink(temp_file.name)
        else:
            print("File successfully written.")

def combine_monthly_pattern_home_panel_summary(data_dir, extend='.csv.gz'):
    print(os.path.join(data_dir, f"*{extend}"))
    all_files = glob(os.path.join(data_dir, f"*{extend}"))

    df = pd.concat([pd.read_csv(f) for f in tqdm(all_files)])

    return df
    
    # print(all_files)

# Merge CSV files
# home_panel_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary'
# home_panel_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary_2023-11-23'
# df = combine_monthly_pattern_home_panel_summary(data_dir=home_panel_dir)
# df.to_csv(r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv', index=False)
# df

In [18]:

# Spend patterns:
# url = r'https://app.deweydata.io/external-api/v3/products/eb6e748a-0fdd-4bc7-9dd7-bbed0890948d/files'
# # save_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 2019
# save_dir = r'D:\SafeGraph\Advan_2023_API\Spend_Patterns'  # Lenova 2018

# POI
url = r'https://app.deweydata.io/external-api/v3/products/63eeb9d8-8439-42b3-a75e-2d9cbdf64cb8/files'
save_dir = r'D:\SafeGraph\Advan_2023_API\POI'  # Lenova 2018


# neighborhood patterns:
# url = r'https://app.deweydata.io/external-api/v3/products/2dfcb598-6e30-49f1-bdba-1deae113a951/files'
# save_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 2019
# save_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'  # Lenova 2018

# monthly patterns:
# url = r"https://app.deweydata.io/external-api/v3/products/5acc9f39-1ca6-4535-b3ff-38f6b9baf85e/files"
# save_dir = r'K:\SafeGraph\Advan_2023_API\Monthly_Patterns'

# # weekly patterns:
# url = r"https://app.deweydata.io/external-api/v3/products/176f0262-c1f6-4dbe-be43-6a6eb21bcf8a/files"
# save_dir = r'K:\SafeGraph\Advan_2023_API\Weekly_Patterns'

# # hourly weather:
# url = r"https://app.deweydata.io/external-api/v3/products/a6be0385-8820-4943-a509-6eac4154b4f6/files"

# monthly home panel summary
# url = r'https://app.deweydata.io/external-api/v3/products/8546740c-b0e9-4556-abb9-4bea2cca9ac9/files'
# save_dir = r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary'


os.makedirs(save_dir, exist_ok=True)
# set key and API endpoint variables
API_KEY = access_token
PRODUCT_API_PATH = url

In [19]:
# df.to_csv(r'F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv', index=False)

In [20]:
results = requests.get(url=PRODUCT_API_PATH,
                                   params={'page': 1,
                                           # 'partition_key_after': '2019-01-01',   # optionally set date value here
                                           # 'partition_key_before': '2024-01-01', 
                                          }, # optionally set date value here
                                   headers={'X-API-KEY': API_KEY,
                                            'accept': 'application/json'
                                           })
response_json = results.json()
# response_json

In [ ]:
response_json

In [22]:
def get_all_download_urls():
    total_pages = 1
    current_page = 0
    df_list = []
    while current_page < total_pages:
        results = requests.get(url=PRODUCT_API_PATH,
                                   params={'page': total_pages,
                                           # 'partition_key_after': '2017-01-01',   # optionally set date value here
                                           # 'partition_key_before': '2024-12-31', 
                                          }, # optionally set date value here
                                   headers={'X-API-KEY': API_KEY,
                                            'accept': 'application/json'
                                           })
        response_json = results.json()
        total_pages = response_json['total_pages']
    
        df = pd.DataFrame(response_json)
        df_list.append(df)
        current_page += 1
            
    response_df = pd.concat(df_list)
    
    
    df_list2 = []
    for idx, row in response_df.iterrows():
        # print(row['download_links'])
        df = pd.DataFrame.from_dict([row['download_links']])
        df_list2.append(df)
        # break
        
    url_df = pd.concat(df_list2).sort_values("partition_key")
    return response_df, url_df

response_df, url_df = get_all_download_urls()
url_df

,link,partition_key,file_name,file_extension,file_size_bytes,modified_at
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-0.csv.gz,.csv.gz,146193779,2024-01-07T20:48:47.561Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-1.csv.gz,.csv.gz,146282352,2024-01-07T20:48:47.556Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-2.csv.gz,.csv.gz,154391307,2024-01-07T20:48:47.574Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-3.csv.gz,.csv.gz,167781076,2024-01-07T20:48:47.563Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-4.csv.gz,.csv.gz,149519890,2024-01-07T20:48:47.566Z
...,...,...,...,...,...,...
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-59.csv.gz,.csv.gz,149649486,2024-01-07T20:48:45.446Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-60.csv.gz,.csv.gz,142311650,2024-01-07T20:48:45.436Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-61.csv.gz,.csv.gz,127156461,2024-01-07T20:48:45.435Z
0,https://s3.us-east-005.backblazeb2.com/amplify...,None,Global_Places_POI_Data-62.csv.gz,.csv.gz,151423083,2024-01-07T20:48:45.436Z


In [23]:
# url_df['partition_key'] = url_df['modified_at'].str[:10]
# url_df

In [24]:
downloaded_cnt = 0
for partition_key, df in url_df.groupby('partition_key'):
   
    df = df.reset_index()
    for i, row in df.iterrows():
        try:
            # print(partition_key, i + 1)
            # print("Downloading: ", row['link'])
            date = row['partition_key']
            year = date[:4]
            month = date[5:7]
            day = date[-2:]
            file_dir = os.path.join(save_dir, year, month, day)
            os.makedirs(file_dir, exist_ok=True)
            print("Year, month, day:", year, month, day)
    
            # base_name = row['file_name']
            base_name = row['file_name'][0:24] + row['file_name'][-35:-7] + f"_{i + 1}.csv.gz"
            # print("base_name:", base_name)
            
            file_name = os.path.join(file_dir, base_name)
             
            print(f'Downloading file {file_name}')
        
            if os.path.exists(file_name):
                print(f"File exists, skip! The file is: {file_name} \n")
                continue
        
            # # loop through download links and save to your computer
            data = requests.get(row['link'], stream=True)
            safe_file_write(file_path=file_name, content=data.content)
             
            #  += 1
            downloaded_cnt += 1
            print(f"    Downloaded {i + 1} / {len(df)} files for {partition_key}.")
            print(f"Downloaded {downloaded_cnt} / {len(url_df)} files in total.")
            # print(f"Downloaded {download_count} files in total, Page {page} / {total_pages}.")
            
            print()
        except Exception as e:
            print("Error:", e)
            continue

print("Done.")

Done.


In [25]:
# STOP

# Not used below


SyntaxError: invalid syntax (3138096080.py, line 3)

# Extract POI

In [2]:
data_dir = r"D:\SafeGraph\Advan_2023_API\POI\2024\01\07"
save_dir = r'D:\SafeGraph\Advan_2023_API\POI\2024\01\07\South_Carolina'

os.makedirs(save_dir, exist_ok=True)

In [3]:
all_files = glob(os.path.join(data_dir, "*.gz"))
print("File count:", len(all_files))
print(all_files[0])
print(all_files[-1])

File count: 64
D:\SafeGraph\Advan_2023_API\POI\2024\01\07\Global_Places_POI_Data-0Global_Places_POI_Data-0_1.csv.gz
D:\SafeGraph\Advan_2023_API\POI\2024\01\07\Global_Places_POI_Data-9Global_Places_POI_Data-9_10.csv.gz


In [4]:
countries = ['US']
target_region = ['SC']

In [6]:
df_list = []
# use_cols = ['PLACEKEY', 'TOP_CATEGORY', 'SUB_CATEGORY', 'NAICS_CODE', 'CITY', 'REGION', 'POSTAL_CODE', 'ISO_COUNTRY_CODE']

for idx, f in enumerate(all_files):
    print(f"Processing: {idx + 1} / {len(all_files)}, ", os.path.basename(f))
    # df = pd.read_csv(f, usecols=use_cols)
    df = pd.read_csv(f)
    t_df = df[df['ISO_COUNTRY_CODE'].isin(countries)] 
    # t_df = df[df['NAICS_CODE'].isin(target_NAICS)]
    # t_df = t_df[t_df['CITY'].isin(target_city)]
    t_df = t_df[t_df['REGION'].isin(target_region)]
    mask = np.logical_and(t_df['NAICS_CODE'].astype(str) < "623", t_df['NAICS_CODE'].astype(str) > "619" ) 
    t_df = t_df[mask]
    
    df_list.append(t_df)
    # break

df_all = pd.concat(df_list)
fname = os.path.join(save_dir, 'South_Carolina_POI_20240107.csv')
df_all.to_csv(fname, index=False)
# print("Saved at:", fname)
df_all

Processing: 1 / 64,  Global_Places_POI_Data-0Global_Places_POI_Data-0_1.csv.gz
Processing: 2 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-10_11.csv.gz
Processing: 3 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-11_12.csv.gz
Processing: 4 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-12_13.csv.gz
Processing: 5 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-13_14.csv.gz
Processing: 6 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-14_15.csv.gz
Processing: 7 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-15_16.csv.gz
Processing: 8 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-16_17.csv.gz
Processing: 9 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-17_18.csv.gz
Processing: 10 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-18_19.csv.gz
Processing: 11 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-19_20.csv.gz
Processing: 12 / 64,  Global_Places_POI_Data-1Global_Places_POI_Data-1_2.csv.gz
Processing: 13 / 64,  Global_

,PLACEKEY,PARENT_PLACEKEY,LOCATION_NAME,SAFEGRAPH_BRAND_IDS,BRANDS,STORE_ID,TOP_CATEGORY,SUB_CATEGORY,NAICS_CODE,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,REGION,POSTAL_CODE,ISO_COUNTRY_CODE,PHONE_NUMBER,OPEN_HOURS,CATEGORY_TAGS,OPENED_ON,CLOSED_ON,TRACKING_CLOSED_SINCE,GEOMETRY_TYPE,DOMAINS,WEBSITE
598,zzy-222@646-66n-jjv,225-224@646-66p-k2k,"Susan Mclean Lpc, Ncc, Cch",NaN,NaN,NaN,Offices of Physicians,"Offices of Physicians, Mental Health Specialists",621112.0,33.787113,-78.999080,172 Waccamaw Medical Park Ct,Conway,SC,29526,US,1.843457e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
692,zzy-222@8gc-mfq-mtv,223-222@8gc-mfq-mtv,Dr Shayla Coomer Llc,NaN,NaN,NaN,Offices of Physicians,Offices of Physicians,62111.0,32.910837,-80.667473,1150 Hungryneck Blvd,Walterboro,SC,29488,US,NaN,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
1200,zzw-222@8gh-5rv-qfz,NaN,Sandra E Hennies MED,NaN,NaN,NaN,Offices of Other Health Practitioners,Offices of Mental Health Practitioners (except...,621330.0,33.994831,-80.970300,906 Burwell Ln,Columbia,SC,29205,US,1.803787e+10,"{ ""Mon"": [], ""Tue"": [[""8:00"", ""19:00""]], ""Wed""...",Mental Health,NaN,NaN,2019-07,POLYGON,"[""sandraehennies.com""]",https://www.sandraehennies.com
1724,222-229@646-5pm-cyv,NaN,Tara Smith,NaN,NaN,NaN,Offices of Other Health Practitioners,Offices of All Other Miscellaneous Health Prac...,621399.0,33.618352,-78.993203,1945 Glenns Bay Rd,Surfside Beach,SC,29575,US,1.843650e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
3645,227-222@8gg-jdb-zxq,NaN,Wade D Reynolds D O Llc,NaN,NaN,NaN,Offices of Physicians,Offices of Physicians,62111.0,33.989986,-81.246525,111 Ivy Park Ln,Lexington,SC,29072,US,NaN,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472409,225-226@8gh-55g-d5f,225-223@8gh-55g-d5f,Mary Woods Lpc,NaN,NaN,NaN,Offices of Physicians,"Offices of Physicians, Mental Health Specialists",621112.0,34.093911,-80.924526,9153 Two Notch Rd Ste C11,Columbia,SC,29223,US,1.803339e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
1472708,222-24z@8gh-5r6-gff,NaN,John Bullard Jr. MD,NaN,NaN,NaN,Offices of Physicians,Offices of Physicians (except Mental Health Sp...,621111.0,34.030033,-81.028282,3209 Colonial Dr,Columbia,SC,29203,US,1.803435e+10,NaN,NaN,NaN,NaN,2019-07,POLYGON,[],NaN
1473397,22d-222@8gg-sp8-zpv,NaN,Complete Pro Home Care,NaN,NaN,NaN,Home Health Care Services,Home Health Care Services,621610.0,34.977153,-81.950459,102 Metro Dr Ste 9,Spartanburg,SC,29303,US,1.864310e+10,"{ ""Mon"": [[""9:00"", ""14:30""]], ""Tue"": [[""9:00"",...",NaN,NaN,NaN,2019-07,POLYGON,"[""completeprohomecare.com""]",http://completeprohomecare.com
1473863,222-22d@8gg-sz2-wzf,222-225@8gg-sz2-wzf,Nigh Steven C,NaN,NaN,NaN,Offices of Other Health Practitioners,Offices of Optometrists,621320.0,34.851861,-82.336418,765 Haywood Rd,Greenville,SC,29607,US,1.864244e+10,NaN,Optometrists,NaN,NaN,2019-07,POLYGON,"[""eyeexamtoday.com""]",http://eyeexamtoday.com/


In [ ]:
df_all

In [6]:
# Do not merge
df_list = []
# use_cols = ['PLACEKEY', 'TOP_CATEGORY', 'SUB_CATEGORY', 'NAICS_CODE', 'CITY', 'REGION', 'POSTAL_CODE', 'ISO_COUNTRY_CODE']

for idx, f in enumerate(all_files):
    print(f"Processing: {idx + 1} / {len(all_files)}, ", os.path.basename(f))
    # df = pd.read_csv(f, usecols=use_cols)
    basename = os.path.basename(f)
    df = pd.read_csv(f)
    t_df = df[df['ISO_COUNTRY_CODE'].isin(countries)] 
    # t_df = df[df['NAICS_CODE'].isin(target_NAICS)]
    # t_df = t_df[t_df['CITY'].isin(target_city)]
    t_df = t_df[t_df['REGION'].isin(target_region)]
    mask = np.logical_and(t_df['NAICS_CODE'].astype(str) < "623", t_df['NAICS_CODE'].astype(str) > "619" ) 
    t_df = t_df[mask]
    
    fname = os.path.join(save_dir, basename)
    t_df.to_csv(fname, index=False)
    
    # df_list.append(t_df)
    break

# df_all = pd.concat(df_list)

# print("Saved at:", fname)
# df_all

Processing: 1 / 64,  Global_Places_POI_Data-0Global_Places_POI_Data-0_1.csv.gz


# Help topic in Deweydata community

https://community.deweydata.io/t/residing-device-count-in-2023-07-and-2023-08-surged-abnomorally/26675